In [ ]:
import pandas as pd
import urllib
import json
import os
from pathlib import Path
import sys
from constants import EVALUATION_DATE_FOLDER

# Evaluate (Deprecated!)

In [3]:
# Load a WAVE API key stored in a local file
with open('../input/api.lab.key', 'r') as f:
    API_KEY = f.read()

## Data Portals

In [30]:
"""
Get filtered resources' page URLs and page IDs
"""
df_pages = pd.read_csv(os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'data-portal_pages.csv'))
df_map = pd.read_csv(os.path.join('..', 'output', 'data-portal_id_map.csv'))

# Get ids to filter by. Let's just look at the manually collected ones for now.
df_filtered = pd.read_csv(os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'data-portal_filtered_ids.csv'))
FILTER_IDS = list(set(df_filtered.id.values.tolist()))

# Filter pages by selected IDs. Also, empty URLs are excluded.
df_pages = df_pages[(df_pages.id.isin(FILTER_IDS)) & (~df_pages.url.isnull())]

# df_pages = df_pages.head(1) # for debuging purposes
df_pages

,id,page_type,page_id,url
0,1,home,1_home,https://david.ncifcrf.gov
1,2,home,2_home,https://www.kegg.jp
2,3,home,3_home,https://www.cbioportal.org/
3,4,home,4_home,https://string-db.org/
4,5,home,5_home,https://www.encodeproject.org/
...,...,...,...,...
6609,58,documentation,58_documentation,https://docs.gsea-msigdb.org/#GSEA/GSEA_FAQ/
6610,59,search,59_search,http://www.cellchat.org/cellchatdb/
6615,60,search_result,60_search_result,http://smart.embl.de/smart/search.cgi?keywords...
6616,60,data_entity,60_data_entity,http://smart.embl.de/smart/show_motifs.pl?ID=F...


In [4]:
"""
Collect raw reports all together first using API calls
"""
def collect_raw_reports_and_save(df_pages, RAW_REPORTS_FOLDER):    
    # Create a folder to store raw reports, if missing
    Path(RAW_REPORTS_FOLDER).mkdir(exist_ok=True)

    """ 
    Collect missing reports one by one, and save them as a file
    """
    for _, row in df_pages.iterrows():
        page_id = row.page_id
        url = row.url

        # Skip if the report already exists
        PAGE_REPORT_PATH = os.path.join(RAW_REPORTS_FOLDER, f'{page_id}.json')
        is_exist = os.path.isfile(PAGE_REPORT_PATH)

        if is_exist:
            print(f'Report for {url} already exists. Skipping ...')
            continue
        else:
            # Twick...
            if int(page_id.split('_')[0]) < 27955:
                PAGE_REPORT_PATH = os.path.join(RAW_REPORTS_FOLDER, f'{int(page_id.split("_")[0]) + 27955}_{page_id.split("_")[1]}.json')
                if os.path.isfile(PAGE_REPORT_PATH):
                    continue
        
        # Refer to https://wave.webaim.org/api/docs#!/request/getRequest for the API documentation
        API_URL = f'https://wave.webaim.org/api/request?key={API_KEY}&reporttype=2&url={url}'
        
        print(f'Retrieving {url} ...')

        try:
            with urllib.request.urlopen(API_URL) as f:
                new_report = json.load(f) # Refer to `../output/raw-reports-examples` to understand the structure of the report

                # Save the raw report
                with open(PAGE_REPORT_PATH, 'w') as f:
                    json.dump(new_report, f)
        except:
            print('Failed!')

In [5]:
RAW_REPORTS_FOLDER = os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'raw-reports', 'data-portal')

In [ ]:
collect_raw_reports_and_save(
    df_pages,
    RAW_REPORTS_FOLDER
)

In [11]:
def create_dataframe_from_raw_reports(df_pages, RAW_REPORTS_FOLDER):
    # Assumption in this function is that the raw reports are already collected

    # All reports to be saved as a dataframe
    reports = []
    for _, row in df_pages.iterrows():
        # The dictionary to be saved as a row in the dataframe
        cleaned_report = {}

        # The primary key column we use to identify each page
        page_id = row.page_id
        
        PAGE_REPORT_PATH = os.path.join(RAW_REPORTS_FOLDER, f'{page_id}.json')

        if not os.path.isfile(PAGE_REPORT_PATH):
            # Twick...
            if int(page_id.split('_')[0]) < 27955:
                PAGE_REPORT_PATH = os.path.join(RAW_REPORTS_FOLDER, f'{int(page_id.split("_")[0]) + 27955}_{page_id.split("_")[1]}.json')
                if not os.path.isfile(PAGE_REPORT_PATH):
                    print(f'Odd. Report for {page_id} does not exist. Skipping ...')
                    continue

        with open(PAGE_REPORT_PATH, 'r') as f:
            page_report = json.load(f)

        is_success = page_report.get('status').get('success')

        # Add basic information
        cleaned_report |= { 
            'page_id': page_id,
            'is_success': is_success
        }


        if is_success == True:
            error_items = page_report.get('categories').get('error').get('items')
            alert_items = page_report.get('categories').get('alert').get('items')
            contrast_items = page_report.get('categories').get('contrast').get('items')
        
            error_count = page_report.get('categories').get('error').get('count')
            alert_count = page_report.get('categories').get('alert').get('count')
            contrast_count = page_report.get('categories').get('contrast').get('count')
            
            cleaned_report |= { 
                'count': error_count + alert_count + contrast_count,
                'category_count': len(error_items) + len(alert_items) + len(contrast_items),
                'error_count': error_count,
                'alert_count': alert_count,
                'contrast_count': contrast_count,
            }

            # Example: 
            # {
            #   'label_missing': {'id': 'label_missing', 'description': 'Missing form label', 'count': 1}, 
            #   'language_missing': {'id': 'language_missing', 'description': 'Language missing or invalid', 'count': 1}, 
            #   'button_empty': {'id': 'button_empty', 'description': 'Empty button', 'count': 1}
            # }

            if len(error_items) != 0:
                for item in error_items.values():
                    cleaned_report |= { f"error_{item.get('id')}": item.get('count') }
            if len(alert_items) != 0:
                for item in alert_items.values():
                    cleaned_report |= { f"alert_{item.get('id')}": item.get('count') }
            if len(contrast_items) != 0:
                for item in contrast_items.values():
                    cleaned_report |= { f"contrast_{item.get('id')}": item.get('count') }
            
            reports.append(cleaned_report)

    return pd.DataFrame.from_records(reports)    

In [34]:
df_reports = create_dataframe_from_raw_reports(df_pages, RAW_REPORTS_FOLDER)
df_reports.to_csv(os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'data-portal_evaluation.csv'), index=False)
df_reports

Odd. Report for 85_home does not exist. Skipping ...
Odd. Report for 1002_home does not exist. Skipping ...
Odd. Report for 1618_home does not exist. Skipping ...
Odd. Report for 3005_home does not exist. Skipping ...
Odd. Report for 3283_home does not exist. Skipping ...
Odd. Report for 3329_home does not exist. Skipping ...
Odd. Report for 3474_home does not exist. Skipping ...
Odd. Report for 3699_home does not exist. Skipping ...
Odd. Report for 3780_home does not exist. Skipping ...
Odd. Report for 3783_home does not exist. Skipping ...
Odd. Report for 3784_home does not exist. Skipping ...
Odd. Report for 3785_home does not exist. Skipping ...
Odd. Report for 3786_home does not exist. Skipping ...
Odd. Report for 3787_home does not exist. Skipping ...
Odd. Report for 3788_home does not exist. Skipping ...
Odd. Report for 3789_home does not exist. Skipping ...
Odd. Report for 3790_home does not exist. Skipping ...
Odd. Report for 3791_home does not exist. Skipping ...
Odd. Report 

,page_id,is_success,count,category_count,error_count,alert_count,contrast_count,error_label_missing,error_language_missing,error_button_empty,...,alert_flash,error_alt_map_missing,error_label_empty,alert_html5_video_audio,error_alt_input_missing,alert_link_word,alert_longdesc,error_marquee,error_blink,alert_applet
0,1_home,True,68,9,3,28,37,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2_home,True,9,6,2,5,2,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3_home,True,2108,16,835,230,1043,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4_home,True,112,8,48,37,27,46.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5_home,True,9,5,4,2,3,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4456,58_documentation,True,7,4,5,2,0,1.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4457,59_search,True,8,7,4,4,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4458,60_search_result,True,70,7,2,58,10,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4459,60_data_entity,True,33,7,3,18,12,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Journals

In [5]:
"""
Get filtered resources' page URLs and page IDs
"""
df_pages = pd.read_csv(os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'journal-portal_pages.csv'))
df_map = pd.read_csv(os.path.join('..', 'output', 'journal-portal_id_map.csv'))

# Get ids to filter by. Let's just look at the manually collected ones for now.
# TODO: use the filtered data from `02-Filter.ipynb`!
df_filtered = pd.read_csv(os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'journal-portal_filtered_ids.csv'))
FILTER_IDS = list(set(df_filtered.id.values.tolist()))

# Filter pages by selected IDs. Also, empty URLs are excluded.
df_pages = df_pages[(df_pages.id.isin(FILTER_IDS)) & (~df_pages.url.isnull())]

# df_pages = df_pages.head(1) # for debuging purposes
df_pages

,id,page_type,page_id,url
0,1,home,1_home,https://onlinelibrary.wiley.com/journal/15424863
2,3,home,3_home,https://www.nature.com/nrm/
3,4,home,4_home,https://www.cell.com/cell/home
4,5,home,5_home,https://www.nejm.org/
5,6,home,6_home,https://www.nature.com/nm/
...,...,...,...,...
28055,179,home,179_home,https://rupress.org/jem
28056,179,research_article,179_research_article,https://rupress.org/jem/article/221/1/e2023092...
28057,179,none_research_article,179_none_research_article,https://rupress.org/jem/article/221/1/e2023019...
28058,179,article_search_result,179_article_search_result,https://rupress.org/jem/search-results?page=1&...


In [6]:
RAW_REPORTS_FOLDER = os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'raw-reports', 'journal-portal')

In [7]:
collect_raw_reports_and_save(
    df_pages,
    RAW_REPORTS_FOLDER
)

Report for https://www.nature.com/nrm/ already exists. Skipping ...
Report for https://www.cell.com/cell/home already exists. Skipping ...
Report for https://www.nejm.org/ already exists. Skipping ...
Report for https://www.nature.com/nm/ already exists. Skipping ...
Report for https://www.nature.com/nbt/ already exists. Skipping ...
Report for https://www.nature.com/nrc/ already exists. Skipping ...
Report for http://www.nature.com/nature/index.html already exists. Skipping ...
Report for https://www.nature.com/ng/ already exists. Skipping ...
Report for https://www.thelancet.com/journals/lancet/home already exists. Skipping ...
Report for http://www.sciencemag.org/magazine.dtl already exists. Skipping ...
Report for https://ascopubs.org/journal/jco already exists. Skipping ...
Report for https://academic.oup.com/nar already exists. Skipping ...
Report for https://www.ahajournals.org/journal/circ already exists. Skipping ...
Report for http://www.cell.com/neuron/home already exists. S

In [12]:
df_reports = create_dataframe_from_raw_reports(df_pages, RAW_REPORTS_FOLDER)
df_reports.to_csv(os.path.join('..', 'output', EVALUATION_DATE_FOLDER, 'journal-portal_evaluation.csv'), index=False)

Odd. Report for 25371_home does not exist. Skipping ...
